In [1]:
import numpy as np, pandas as pd
import train as t
from src import helper_models as hm, helper_data as hd
from importlib import reload
import keras




Using TensorFlow backend.
/home/marco/.conda/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/marco/.conda/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/marco/.conda/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/marco/.conda/envs/gpu/lib/python3.7


Running /home/marco/.conda/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py ['-f', '/home/marco/.local/share/jupyter/runtime/kernel-768ce432-92da-49de-9df8-f3d963c4befc.json']


In [7]:
# Reload imports of custom modules, in case I am modifying and reloading them live
reload(t)
reload(hd)
reload(hm)



Running /home/marco/.conda/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py ['-f', '/home/marco/.local/share/jupyter/runtime/kernel-40abbda4-1023-49da-8446-0c1b2203dccb.json']


<module 'src.helper_models' from '/home/marco/Dropbox/PROJECTS/MACHINE-LEARNING/AQUASCOPE/plankifier/src/helper_models.py'>

# Initialize simulation

In [2]:
sim=t.Ctrain()

default
Namespace(L=128, aug=False, bs=32, class_select=None, datakind='mixed', datapath='./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28//', initial_epoch=0, layers=[256, 128], lr=5e-05, model='mlp', opt='sgd', outpath='./out//', plot=False, testSplit=0.2, totEpochs=5, ttkind='mixed', verbose=False)


# Init parameters

In [3]:
# parameters are set upon class init. Here we set them again as an example

sim.SetParameters('default') 

# The following is to show that parameters get automatically updated (see next cell)
print("\nDefault selection of classes (None is all classes):",sim.params.class_select)

Namespace(L=128, aug=False, bs=32, class_select=None, datakind='mixed', datapath='./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28//', initial_epoch=0, layers=[256, 128], lr=5e-05, model='mlp', opt='sgd', outpath='./out//', plot=False, testSplit=0.2, totEpochs=5, ttkind='mixed', verbose=False)

Default selection of classes (None is all classes): None


# Load data

In [4]:
smaller_class_select=['chaoborus','bosmina','unknown_plankton']
sim.LoadData(L=32, class_select=smaller_class_select, datakind='mixed')

# The class_select got updated, along with L and datakind
print("\nDefault selection of classes (None is all classes):",sim.params.class_select)

chaoborus
bosmina
unknown_plankton

Default selection of classes (None is all classes): ['chaoborus', 'bosmina', 'unknown_plankton']


# Create sets (test and train)

In [5]:
# sim.CreateTrainTestSets()

tt=t.CTrainTestSet(sim.data.X, sim.data.y, ttkind='feat')
tt.Split()




# Create Model

In [7]:
checkpointer    = keras.callbacks.ModelCheckpoint(filepath=sim.params.outpath+'/bestweights.hdf5', monitor='val_loss', verbose=0, save_best_only=True) # save the model at every epoch in which there is an improvement in test accuracy
coitointerrotto = keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
logger          = keras.callbacks.callbacks.CSVLogger(sim.params.outpath+'epochs.log', separator=' ', append=False)

trainParams=hm.CreateParams(
                            layers=sim.params.layers, 
                            lr=sim.params.lr,
                            bs=sim.params.bs,
                            totEpochs=sim.params.totEpochs,
                            callbacks= [checkpointer, logger, coitointerrotto]
                            )

#Don't run both the following lines
# If model is not mixed
history, model = hm.MLP(tt.trainX, tt.trainY, tt.testX, tt.testY, trainParams)
# If model is mixed
history, model = hm.MixedModel([self.tt.trainXimage,self.tt.trainXfeat], self.tt.trainY, [self.tt.testXimage,self.tt.testXfeat], self.tt.testY, trainParams)



Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 0s 1ms/step - loss: 1.4487 - accuracy: 0.4875 - val_loss: 1.6241 - val_accuracy: 0.4000
Epoch 2/5
80/80 [==============================] - 0s 75us/step - loss: 1.4467 - accuracy: 0.4875 - val_loss: 1.6217 - val_accuracy: 0.4000
Epoch 3/5
80/80 [==============================] - 0s 114us/step - loss: 1.4447 - accuracy: 0.4875 - val_loss: 1.6194 - val_accuracy: 0.4000
Epoch 4/5
80/80 [==============================] - 0s 100us/step - loss: 1.4429 - accuracy: 0.4875 - val_loss: 1.6173 - val_accuracy: 0.4000
Epoch 5/5
80/80 [==============================] - 0s 96us/step - loss: 1.4411 - accuracy: 0.4875 - val_loss: 1.6149 - val_accuracy: 0.4000
